In [ ]:
# need ipykernel > 6
# ! pip install ipykernel==6.23.1
! pip install git+https://github.com/ginihumer/Amumo.git

In [ ]:
from amumo import model
from amumo import data

In [ ]:
dataset = data.DiffusionDB_Dataset(path="2m_first_1k")
def get_data_helper(dataset, filters=[], method=any):
    all_images, all_prompts = dataset.get_filtered_data(filters, method=method)
    print(len(all_images))

    dataset_name = dataset.name
    if len(filters) > 0:
        dataset_name = dataset_name + '_filter-' + method.__name__ + '_' + '-'.join(filters)
    else:
        dataset_name = dataset_name + '_size-%i'%len(all_images)

    return all_images, all_prompts, dataset_name

all_images, all_prompts, dataset_name = get_data_helper(dataset, filters=[], method=any) # filters = ["dog"], method=all
dataset_name

In [ ]:
class CustomModel(model.CLIPModelInterface):
    available_models = clip.available_models()
    model_name = 'CLIP'

    def __init__(self, name='RN50', device='cpu') -> None:
        super().__init__(name, device)
        self.model, self.preprocess = clip.load(name, device=device)
        self.model.eval()
        self.logit_scale = self.model.logit_scale

    def encode_image(self, images):
        images = [self.preprocess(i) for i in images]
        image_input = torch.tensor(np.stack(images)).to(self.device)
        return self.model.encode_image(image_input).float().cpu()

    def encode_text(self, texts):
        text_tokens = clip.tokenize(texts, truncate = True).to(self.device)
        return self.model.encode_text(text_tokens).float().cpu()

In [ ]:
from amumo.widgets import CLIPExplorerWidget


clipexplorer_widget = CLIPExplorerWidget(dataset_name, all_images, all_prompts, models=[CustomModel()])
clipexplorer_widget